## Read data

In [1]:
import pandas as pd
from neo4j import GraphDatabase

In [2]:
data = pd.read_json('../Data/ScrapedData.json')

In [3]:
data.head()

,edges,got_type,name,node_type,properties,redirected_urls,status,url
0,"[{'Source': '/wiki/Faceless_Men', 'Label': 'Se...","[Religious cult, Assassin guild]",Faceless Men,Organization,"[Season(s), Mentioned in, Status, Type, Notabl...",None,[Active],/wiki/Faceless_Men
1,"[{'Source': '/wiki/Old_Gods_of_the_Forest', 'L...","[Polytheistic, Animistic, Henotheistic]",Old Gods of the Forest,Organization,"[Type, Clergy, Location, Leader, Center]",None,NaN,/wiki/Old_Gods_of_the_Forest
2,"[{'Source': '/wiki/R%27hllor', 'Label': 'Clerg...",[Ditheism],"R'hllor, the Lord of Light",Organization,"[Type, Clergy, Location, Leader, Notable Membe...",None,NaN,/wiki/R%27hllor
3,"[{'Source': '/wiki/Drowned_God', 'Label': 'Cle...",[Monolatristic],Drowned God,Organization,"[Type, Clergy, Location, Notable Members, Form...",None,NaN,/wiki/Drowned_God
4,"[{'Source': '/wiki/Unsullied', 'Label': 'Seaso...",[Spear infantry],Unsullied,Organization,"[Season(s), Mentioned in, Status, Type, Allegi...",None,[Active],/wiki/Unsullied


In [4]:
data.node_type.unique()

array(['Organization', 'Person', 'Event', 'Episode', 'Animal', 'Location',
       'HistoriesNLore', 'Weapon', 'House', 'PersonType', 'Religion',
       'Season'], dtype=object)

## Extract edges

In [5]:
# Create a dictionary for redirected_urls

red = data[data.redirected_urls.isnull() == False]
red[red.redirected_urls.apply(len) == 1]
redirect_dict = {}
red.redirected_urls = red.redirected_urls.apply(lambda x: x[0])

redirect_dict = dict(zip(red.redirected_urls,red.url))


C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\pandas\core\generic.py:4401: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [6]:
edges = []
def process_edges(arr):    
    for json in arr:
        edges.append([json['Source'], json['Label'], json['Target']])
    pass

data.edges.apply(process_edges)
dfe = pd.DataFrame(edges, columns=['Source', 'Label', 'Target'])

dfe.head()


,Source,Label,Target
0,/wiki/Faceless_Men,Season(s),/wiki/Season_2
1,/wiki/Faceless_Men,Season(s),/wiki/Season_5
2,/wiki/Faceless_Men,Season(s),/wiki/Season_6
3,/wiki/Faceless_Men,Mentioned in,/wiki/Valar_Morghulis
4,/wiki/Faceless_Men,Notable Members,/wiki/Jaqen_H%27ghar


In [7]:
# check is all source is in nodes data
print('No. of unique Sources in edges data = ' + str(dfe.Source.unique().__len__()))
print('No. of unique nodes in nodes data = ' + str(data.url.unique().__len__()))

print('No. of nodes with 0 outgoing edges = ')
data[data.edges.apply(len) == 0]


No. of unique Sources in edges data = 2190
No. of unique nodes in nodes data = 2192
No. of nodes with 0 outgoing edges = 


,edges,got_type,name,node_type,properties,redirected_urls,status,url
416,[],NaN,House Wells,House,[],None,NaN,/wiki/House_Wells
1641,[],NaN,Black Friday,Event,"[Beginning, End, Place, Outcome]",None,NaN,/wiki/Black_Friday


## Remove edges where target node is not a valid node

In [8]:
# remove edges where target not in nodes data
print('No. of unique Target nodes in edges data(before) = ' + str(dfe.Target.unique().__len__()))

valid_nodes = set(data.url)

def get_redirected_target(url):
    if url in valid_nodes:
        return url
    elif url in redirect_dict.keys():
        return redirect_dict[url]
    else:
        return None

dfe['Target'] = dfe.Target.apply(get_redirected_target)
dfe[dfe.Target.isnull()]

print('No. of unique Target nodes in edges data(after)  = ' + str(dfe.Target.unique().__len__()))
# remove all None target nodes
# Edges is not ready

dfe = dfe[dfe.Target.isnull() == False]

dfe.head()

No. of unique Target nodes in edges data(before) = 2877
No. of unique Target nodes in edges data(after)  = 1560


,Source,Label,Target
0,/wiki/Faceless_Men,Season(s),/wiki/Season_2
1,/wiki/Faceless_Men,Season(s),/wiki/Season_5
2,/wiki/Faceless_Men,Season(s),/wiki/Season_6
3,/wiki/Faceless_Men,Mentioned in,/wiki/Valar_Morghulis
4,/wiki/Faceless_Men,Notable Members,/wiki/Jaqen_H%27ghar


## Process data- remove 'wiki' & commas

In [9]:
print('remove /wiki/ part from the urls-')
dfe.Source = dfe.Source.apply(lambda x: x[6:])
dfe.Target = dfe.Target.apply(lambda x: x[6:])
data.url = data.url.apply(lambda x: x[6:])

remove /wiki/ part from the urls-


In [10]:
#replace comma with semi-colon before writing to csv
def get_formatted_value(arr):
    if isinstance(arr, list) and len(arr) > 0:
        return '; '.join(arr).replace(',', '-')
    return None
data.got_type = data.got_type.apply(get_formatted_value)
data.status = data.status.apply(get_formatted_value)


In [11]:
data.drop(['edges', 'properties', 'redirected_urls'], axis=1, inplace=True)

data.head()

,got_type,name,node_type,status,url
0,Religious cult; Assassin guild,Faceless Men,Organization,Active,Faceless_Men
1,Polytheistic; Animistic; Henotheistic,Old Gods of the Forest,Organization,None,Old_Gods_of_the_Forest
2,Ditheism,"R'hllor, the Lord of Light",Organization,None,R%27hllor
3,Monolatristic,Drowned God,Organization,None,Drowned_God
4,Spear infantry,Unsullied,Organization,Active,Unsullied


## Process edges- add source_node_type & target_node_type

In [12]:
dfe = pd.merge(dfe, data[['url', 'node_type']], how='left', left_on='Source', right_on='url')[['Source', 'Label', 'Target', 'node_type']]
dfe.columns = ['Source', 'Label', 'Target', 'Source_Node_Type']
dfe.head()

dfe = pd.merge(dfe, data[['url', 'node_type']], how='left', left_on='Target', right_on='url')[['Source', 'Label', 'Target', 'Source_Node_Type', 'node_type']]
dfe.columns = ['Source', 'Label', 'Target', 'Source_Node_Type', 'Target_Node_Type']
dfe.head()

,Source,Label,Target,Source_Node_Type,Target_Node_Type
0,Faceless_Men,Season(s),Season_2,Organization,Season
1,Faceless_Men,Season(s),Season_5,Organization,Season
2,Faceless_Men,Season(s),Season_6,Organization,Season
3,Faceless_Men,Mentioned in,Valar_Morghulis,Organization,Episode
4,Faceless_Men,Notable Members,Jaqen_H%27ghar,Organization,Person


## Store all entity data

In [13]:
data.to_csv('../Data/GOT_Entities.csv', index=False)

## Analyse relationships

### Remove unwanted relationships

In [14]:
unwanted_rels = ['Referred', 'Titles', 'Age', 'Portrayed by', 'Date', 'Date of birth', 'Family', 
                'Civilian Casualties', # only 6
                 'Date of Founding', #1
                 'Also known as', #6
                 'Date of founding', #6
                 'Date of dissolution', #10
                ]
dfe = dfe[(dfe.Label.apply(lambda x: x not in unwanted_rels))]
dfe.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13616 entries, 0 to 16503
Data columns (total 5 columns):
Source              13616 non-null object
Label               13616 non-null object
Target              13616 non-null object
Source_Node_Type    13616 non-null object
Target_Node_Type    13616 non-null object
dtypes: object(5)
memory usage: 638.2+ KB


### Type of Labels

In [15]:
dfe[['Source_Node_Type', 'Target_Node_Type', 'Label', 'Source']].groupby(by=['Label'], as_index=False).count().sort_values(by='Source', ascending=True)

,Label,Source_Node_Type,Target_Node_Type,Source
34,Material,1,1,1
25,Habitat,1,1,1
4,Beginning,2,2,2
16,End,2,2,2
61,Sigil,3,3,3
24,Geography,3,3,3
45,Place of origin,4,4,4
11,Clergy,4,4,4
68,Villages,5,5,5
6,Capital,5,5,5


### Merge/Delete few Labels

In [16]:
dfe.loc[dfe.Label == 'Regional capital', 'Label'] = 'Capital'
dfe.loc[dfe.Label == 'Predecessors', 'Label'] = 'Predecessor'
dfe.loc[dfe.Label == 'Successors', 'Label'] = 'Successor'
dfe.loc[dfe.Label == 'Vassals', 'Label'] = 'Allegiance'

filter_remove_labels = (dfe.Label=='Habitat')|(dfe.Label=='Material')|(dfe.Label=='End')|(dfe.Label=='Beginning')|(dfe.Label=='Geography')|(dfe.Label=='Sigil')
dfe = dfe[~filter_remove_labels]

### P2P relationships

In [17]:
dfe[(dfe.Source_Node_Type == 'Person') & (dfe.Target_Node_Type == 'Person')].Label.unique()

array(['Predecessor', 'Death', 'Father', 'Mother', 'Spouse', 'Siblings',
       'Allegiance', 'Lovers', 'Successor', 'Children', 'Last seen',
       'Appeared in', 'Death shown in episode'], dtype=object)

In [18]:
# dfe[(dfe.Source_Node_Type == 'Person') & (dfe.Target_Node_Type == 'Person') & (dfe.Label == 'Death shown in episode')]
# dfe[(dfe.Source_Node_Type == 'Person') & (dfe.Target_Node_Type == 'Person') & (dfe.Label == 'Last seen')]
# dfe[(dfe.Source_Node_Type == 'Person') & (dfe.Target_Node_Type == 'Person') & (dfe.Label == 'Appeared in')]

remove_filter = (dfe.Source_Node_Type == 'Person') & (dfe.Target_Node_Type == 'Person') & ((dfe.Label == 'Appeared in')|(dfe.Label == 'Last seen')|(dfe.Label == 'Death shown in episode'))

#drop these relationships
dfe = dfe[~remove_filter]

In [19]:
dfe[(dfe.Source_Node_Type == 'Person') & (dfe.Target_Node_Type == 'Person')].Label.unique()

array(['Predecessor', 'Death', 'Father', 'Mother', 'Spouse', 'Siblings',
       'Allegiance', 'Lovers', 'Successor', 'Children'], dtype=object)

### Create Relationships

In [20]:
## Combine relationships
dfe['Relation'] = dfe.Label

In [21]:
print('List of relatinship types:')
dfe.Relation.unique()

List of relatinship types:


array(['Season(s)', 'Mentioned in', 'Notable Members', 'Former Members',
       'Religion', 'Center', 'Location', 'Clergy', 'Allegiance', 'Leader',
       'Founder', 'Appeared in', 'First seen', 'Predecessor', 'Last seen',
       'Death', 'Death shown in episode', 'Culture', 'Conflict', 'Place',
       'Outcome', 'Origin', 'Father', 'Mother', 'Spouse', 'Siblings',
       'Battles', 'Rulers', 'Narrated by', 'Lovers', 'Successor',
       'Children', 'Maker', 'Owner', 'Seat', 'Region', 'Lord', 'Capital',
       'Cities', 'Towns', 'Castles', 'Species', 'Range', 'Ruler',
       'Population', 'Heir', 'Ancestral weapon', 'Places of Note',
       'Formerly', 'Places of note', 'Military', 'Institutions',
       'Villages', 'Place of origin', 'Formed from', 'Cadet branches',
       'Military strength', 'Premiere', 'Finale'], dtype=object)

In [22]:
dfe[['Source_Node_Type', 'Target_Node_Type', 'Relation', 'Source']].groupby(by=['Source_Node_Type', 'Target_Node_Type', 'Relation'], as_index=False).count().sort_values(by='Source', ascending=False)

,Source_Node_Type,Target_Node_Type,Relation,Source
183,Person,Season,Season(s),1418
151,Person,House,Allegiance,893
161,Person,Location,Origin,761
175,Person,Person,Siblings,753
179,Person,PersonType,Culture,500
137,Person,Episode,Mentioned in,400
135,Person,Episode,First seen,380
169,Person,Person,Children,367
132,Person,Episode,Appeared in,359
37,House,House,Allegiance,353


In [23]:
# Merge to same relationship for different labels

filter_seasons = (dfe.Label=='Season(s)')|(dfe.Label=='Appeared in')|(dfe.Label=='Mentioned in')|(dfe.Label=='First seen')|(dfe.Label=='Last seen')|(dfe.Label=='Death shown in episode')
dfe.loc[filter_seasons, ['Relation']] = 'SeenOrMentioned'

filter_member = (dfe.Label=='Notable Members')| (dfe.Label=='Former Members')
dfe.loc[filter_member, ['Relation']] = 'Membership'

dfe.loc[(dfe.Label == 'End'), ['Relation']] = 'Outcome'

filter_location = (dfe.Label=='Region')|(dfe.Label=='Seat')|(dfe.Label=='Origin')
dfe.loc[filter_location, ['Relation']] = 'AssociatedLocation'


In [24]:
dfe.Relation = dfe.Relation.str.replace(' ', '')

In [25]:
print('total edges: ')
print(dfe.count())

print('total unique edges (number converted to weight):')
print(dfe.groupby(by=['Source', 'Target'], as_index=False).count().count())
dfe.head()

total edges: 
Source              13600
Label               13600
Target              13600
Source_Node_Type    13600
Target_Node_Type    13600
Relation            13600
dtype: int64
total unique edges (number converted to weight):
Source              12741
Target              12741
Label               12741
Source_Node_Type    12741
Target_Node_Type    12741
Relation            12741
dtype: int64


,Source,Label,Target,Source_Node_Type,Target_Node_Type,Relation
0,Faceless_Men,Season(s),Season_2,Organization,Season,SeenOrMentioned
1,Faceless_Men,Season(s),Season_5,Organization,Season,SeenOrMentioned
2,Faceless_Men,Season(s),Season_6,Organization,Season,SeenOrMentioned
3,Faceless_Men,Mentioned in,Valar_Morghulis,Organization,Episode,SeenOrMentioned
4,Faceless_Men,Notable Members,Jaqen_H%27ghar,Organization,Person,Membership


## Generate Node files

In [26]:
script_template = ("LOAD CSV WITH HEADERS FROM {0} AS row\n"
                   "CREATE (n:{1})\n"
                   "SET n = row\n\n")

script_index_template = "CREATE INDEX ON :{0}(url)\n\n"

In [27]:
#Write nodes and edges to files
script = []
for label, df in data.groupby(by='node_type'):
    filepath = '../Outputs/Node_' + label + '.csv'
    df[['name', 'node_type', 'got_type', 'status', 'url']].to_csv(filepath, index=False)
    fullfilepath = '"file:///Node_' + label + '.csv"'
    script.append(script_template.format(fullfilepath, label))
    script.append(script_index_template.format(label))


## Generate edge files

In [28]:
dfe.head()

,Source,Label,Target,Source_Node_Type,Target_Node_Type,Relation
0,Faceless_Men,Season(s),Season_2,Organization,Season,SeenOrMentioned
1,Faceless_Men,Season(s),Season_5,Organization,Season,SeenOrMentioned
2,Faceless_Men,Season(s),Season_6,Organization,Season,SeenOrMentioned
3,Faceless_Men,Mentioned in,Valar_Morghulis,Organization,Episode,SeenOrMentioned
4,Faceless_Men,Notable Members,Jaqen_H%27ghar,Organization,Person,Membership


In [29]:
script_edge_template = ("LOAD CSV WITH HEADERS FROM {0} AS row\n"
                   "MERGE (source:{1} {{url: row.Source}})\n"
                   "MERGE (target:{2} {{url: row.Target}})\n"
                   "CREATE (source)-[:{3}]->(target)\n\n")


In [30]:

for s, df in dfe.groupby(by=['Source_Node_Type', 'Target_Node_Type', 'Relation'], as_index=False):
    filepath = '../Outputs/Edge_' + s[0] + s[1] + s[2] + '.csv'
    df[['Source', 'Target', 'Relation', 'Label']].to_csv(filepath, index=False)
    fullfilepath = '"file:///Edge_' + s[0] + s[1] + s[2] + '.csv"'
    script.append(script_edge_template.format(fullfilepath, s[0], s[1], s[2]))
    

## Store nodes & edges of graph

In [31]:
data[['name', 'node_type', 'got_type', 'status', 'url']].to_csv('../Data/nodes.csv', index=False)
dfe.to_csv('../Data/edges.csv', index=False)

In [32]:
f = open("Cypher_Script.txt", "w")
f.write('\n'.join(script))

40082

## Run queries for neo4j

In [ ]:
uri = "bolt://localhost:7687"
driver = GraphDatabase.driver(uri, auth=("neo4j", "1234"))


In [ ]:
for query in script:
    with driver.session() as session:
        session.run(query)